# ** TO DO!!!!!**
- Stratify on the x`s before the train-test-split?
- measure with mae - mean absolute error

# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split

# Loading data

In [ ]:
!pwd

/content


In [ ]:
url = './sale_dataset-most outliers removed.csv'
df = pd.read_csv(url)

# EDA

In [ ]:
df.head()

,area,building_type,num_bath_rooms,price,purpose,locality,id,security_amenity_count,maintenance_or_cleaning_amenity_count,expendable_amenity_count,unclassify_amenity_count
0,1185.0,Apartment,0.0,6100000.0,Sale,Khilgaon,bproperty-0,1,2,2,3
1,1140.0,Apartment,0.0,7500000.0,Sale,Mirpur,bproperty-2,2,2,2,4
2,1445.0,Apartment,3.0,10800000.0,Sale,Banasree,bproperty-4,0,2,1,4
3,1475.0,Apartment,3.0,10500000.0,Sale,Uttara,bproperty-7,2,3,3,4
4,1600.0,Apartment,3.0,13100000.0,Sale,Rampura,bproperty-12,0,2,2,3


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4436 entries, 0 to 4435
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   area                                   4436 non-null   float64
 1   building_type                          4436 non-null   object 
 2   num_bath_rooms                         4436 non-null   float64
 3   price                                  4436 non-null   float64
 4   purpose                                4436 non-null   object 
 5   locality                               4436 non-null   object 
 6   id                                     4436 non-null   object 
 7   security_amenity_count                 4436 non-null   int64  
 8   maintenance_or_cleaning_amenity_count  4436 non-null   int64  
 9   expendable_amenity_count               4436 non-null   int64  
 10  unclassify_amenity_count               4436 non-null   int64  
dtypes: f

In [ ]:
df.isnull().sum()

area                                     0
building_type                            0
num_bath_rooms                           0
price                                    0
purpose                                  0
locality                                 0
id                                       0
security_amenity_count                   0
maintenance_or_cleaning_amenity_count    0
expendable_amenity_count                 0
unclassify_amenity_count                 0
dtype: int64

Feature 

Checking so enough values are in every feature

In [ ]:
# Amount in amenities that have value zero
#zero_relax = (df['relaxation_amenity_count'] == 0).sum()
zero_security = (df['security_amenity_count'] == 0).sum()
zero_maint_clean = (df['maintenance_or_cleaning_amenity_count'] == 0).sum()
#zero_social = (df['social_amenity_count'] == 0).sum()
zero_expend = (df['expendable_amenity_count'] == 0).sum()
#zero_service_staff = (df['service_staff_amenity_count'] == 0).sum()
zero_unclass = (df['unclassify_amenity_count'] == 0).sum()

print("\n Values that ARE zero")
#print("relaxation_amenity_count:", zero_relax)
print("security_amenity_count:", zero_security)

print("maintenance_or_cleaning_amenity_count:", zero_maint_clean)
#print("social_amenity_count:", zero_social)
print("expendable_amenity_count:", zero_expend)
#print("service_staff_amenity_count:", zero_service_staff)
print("unclassify_amenity_count:", zero_unclass)

print("\n Values that are NOT zero")
#print("relaxation_amenity_count:", 4436-zero_relax)
print("security_amenity_count:", 4436-zero_security)
print("maintenance_or_cleaning_amenity_count:", 4436-zero_maint_clean)
#print("social_amenity_count:", 4436-zero_social)
print("expendable_amenity_count:", 4436-zero_expend)
#print("service_staff_amenity_count:", 4436-zero_service_staff)
print("unclassify_amenity_count:", 4436-zero_unclass)


 Values that ARE zero
security_amenity_count: 1556
maintenance_or_cleaning_amenity_count: 419
expendable_amenity_count: 121
unclassify_amenity_count: 0

 Values that are NOT zero
security_amenity_count: 2880
maintenance_or_cleaning_amenity_count: 4017
expendable_amenity_count: 4315
unclassify_amenity_count: 4436


# Splitting, Train, Val, Test


In [ ]:
# Storing features in X and y
# Dropping the target from X
# Dropping purpose, locality, id
# All have purpose = sale, all have locality = Dhaka, id is irrelevant for model training
X = df.drop(['price', 'purpose', 'locality', 'id'], axis=1)

y = df['price']

# Perform train-validation-test split
X_train, X_testval, y_train, y_testval = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval, test_size=0.5, random_state=42)

# Print the sizes of each split
print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Test set size:", X_test.shape[0])


Training set size: 3105
Validation set size: 665
Test set size: 666


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4436 entries, 0 to 4435
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   area                                   4436 non-null   float64
 1   building_type                          4436 non-null   object 
 2   num_bath_rooms                         4436 non-null   float64
 3   security_amenity_count                 4436 non-null   int64  
 4   maintenance_or_cleaning_amenity_count  4436 non-null   int64  
 5   expendable_amenity_count               4436 non-null   int64  
 6   unclassify_amenity_count               4436 non-null   int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 242.7+ KB


# Concatinating train_df


In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)

In [ ]:
val_df = pd.concat([X_val, y_val], axis=1)

In [ ]:
test_df = pd.concat([X_test, y_test], axis=1)

## Checking building_type

In [ ]:
unique_values = train_df['building_type'].unique()
print(unique_values)

['Apartment' 'Duplex']


In [ ]:
apartment_count = train_df[train_df['building_type'] == 'Apartment'].shape[0]
duplex_count = train_df[train_df['building_type'] == 'Duplex'].shape[0]

print("Antal 'Apartment':", apartment_count)
print("Antal 'Duplex':", duplex_count)

Antal 'Apartment': 3104
Antal 'Duplex': 1


## Dropping building_type
- Apartments 3104
- Duplex 1
There is not enough duplex to have relevant information, so dropping it.

In [ ]:
# Dropping building_type from all sets
train_df = train_df.drop('building_type', axis=1)
val_df = val_df.drop('building_type', axis=1)
test_df = test_df.drop('building_type', axis=1)

In [ ]:
# Getting the featurenames
column_names = df.columns.tolist()
print(column_names)

['area', 'building_type', 'num_bath_rooms', 'price', 'purpose', 'locality', 'id', 'security_amenity_count', 'maintenance_or_cleaning_amenity_count', 'expendable_amenity_count', 'unclassify_amenity_count']


# Scaling
- Scaling
- Normalizing


In [ ]:
print(train_df[:2])

        area  num_bath_rooms  security_amenity_count  \
4110  1472.0             4.0                       2   
2757  1100.0             0.0                       0   

      maintenance_or_cleaning_amenity_count  expendable_amenity_count  \
4110                                      1                         3   
2757                                      1                         2   

      unclassify_amenity_count       price  
4110                         3  13500000.0  
2757                         4   5170000.0  


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3105 entries, 4110 to 860
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   area                                   3105 non-null   float64
 1   num_bath_rooms                         3105 non-null   float64
 2   security_amenity_count                 3105 non-null   int64  
 3   maintenance_or_cleaning_amenity_count  3105 non-null   int64  
 4   expendable_amenity_count               3105 non-null   int64  
 5   unclassify_amenity_count               3105 non-null   int64  
 6   price                                  3105 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 194.1 KB


In [ ]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 665 entries, 220 to 1438
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   area                                   665 non-null    float64
 1   num_bath_rooms                         665 non-null    float64
 2   security_amenity_count                 665 non-null    int64  
 3   maintenance_or_cleaning_amenity_count  665 non-null    int64  
 4   expendable_amenity_count               665 non-null    int64  
 5   unclassify_amenity_count               665 non-null    int64  
 6   price                                  665 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 41.6 KB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 666 entries, 1783 to 2117
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   area                                   666 non-null    float64
 1   num_bath_rooms                         666 non-null    float64
 2   security_amenity_count                 666 non-null    int64  
 3   maintenance_or_cleaning_amenity_count  666 non-null    int64  
 4   expendable_amenity_count               666 non-null    int64  
 5   unclassify_amenity_count               666 non-null    int64  
 6   price                                  666 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 41.6 KB


In [ ]:
train_df.columns.tolist()

['area',
 'num_bath_rooms',
 'security_amenity_count',
 'maintenance_or_cleaning_amenity_count',
 'expendable_amenity_count',
 'unclassify_amenity_count',
 'price']

In [ ]:
val_df.columns.tolist()

['area',
 'num_bath_rooms',
 'security_amenity_count',
 'maintenance_or_cleaning_amenity_count',
 'expendable_amenity_count',
 'unclassify_amenity_count',
 'price']

In [ ]:
test_df.columns.tolist()

['area',
 'num_bath_rooms',
 'security_amenity_count',
 'maintenance_or_cleaning_amenity_count',
 'expendable_amenity_count',
 'unclassify_amenity_count',
 'price']

## Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Skapa en instans av StandardScaler
scaler = StandardScaler()

# Skala de kolumnerna i train_df
train_df_scaled = train_df.copy()  # Skapa en kopia av train_df
train_df_scaled = scaler.fit_transform(train_df_scaled)

# Skala de numeriska kolumnerna i X_val
val_df_scaled = val_df.copy()  # Skapa en kopia av X_val
val_df_scaled = scaler.transform(val_df_scaled)

# Skala de numeriska kolumnerna i X_test
test_df_scaled = test_df.copy()  # Skapa en kopia av X_test
test_df_scaled = scaler.transform(test_df_scaled)


### Checking scale

In [ ]:
print(train_df_scaled[:2])

[[ 0.87323146  1.75472064  1.35354275 -0.83577938  0.1698877  -0.6460438
   2.05004418]
 [-0.90857574 -0.99873866 -1.18205317 -0.83577938 -0.69486855  1.03681358
  -0.99823126]]


In [ ]:
print(val_df_scaled[:2])

[[ 1.00734598 -0.99873866  0.08574479  0.332656   -0.69486855 -0.6460438
   1.2083835 ]
 [-0.57328944  0.37799099  1.35354275  1.50109137  0.1698877   1.03681358
  -0.16388936]]


In [ ]:
print(test_df_scaled[:2])

[[ 0.04938512 -0.99873866  1.35354275  1.50109137  0.1698877   1.03681358
  -1.06044096]
 [-1.77074052  0.37799099  0.08574479 -0.83577938 -1.55962479 -0.6460438
  -1.24341068]]


# Splitting X and y from train_df, cal_df, test_df

In [ ]:
# Split the training data into X_train and y_train
X_train = train_df_scaled[:, :-1]  # Exclude the last column, which is 'price'
y_train = train_df_scaled[:, -1]  # Take only the last column, which is 'price'

# Split the validation data into X_val and y_val
X_val = val_df_scaled[:, :-1]
y_val = val_df_scaled[:, -1]

# Split the test data into X_test and y_test
X_test = test_df_scaled[:, :-1]
y_test = test_df_scaled[:, -1]

# Model Selection

## Models:
- RandomForestRegressor
- linear_model
- Lasso
- DecisionTreeRegressor
- Ridge
- GradientBoostingRegressor
- SVR
- XGBRegressor

In [ ]:
# Stratified crossvalidation for proportions in every fold being same proportion as in dataset
stratfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### RandomForestRegressor

In [ ]:
forest_reg = RandomForestRegressor(random_state=42)

hyper_param_grid_rf = [{'n_estimators': [80, 90, 100, 110, 120],
'max_depth': [None, 5, 10, 20],
'min_samples_split': [2, 5, 10],
'max_features': ['auto', 'sqrt', 'log2'],
'min_samples_leaf': [1, 2, 4],
'bootstrap': [True, False]}]



rf_gs = GridSearchCV(forest_reg, hyper_param_grid_rf, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

rf_gs.fit(X_train, y_train)
rf_gs.best_params_

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 90}

In [ ]:
# Get best model
best_model_rf = rf_gs.best_estimator_

In [ ]:
# Measuring from validation
y_pred_rf = best_model_rf.predict(X_val)
mse_rf = mean_squared_error(y_val, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
print("RandomForestRegressor:")
print("RMSE: ", rmse_rf)
print("")

RandomForestRegressor:
RMSE:  0.5553949129916903



In [ ]:
# Measuring from kfold crossvalidation
np.sqrt(-rf_gs.best_score_)

0.5740952795742271

In [ ]:
# Errors
errors = y_val - y_pred_rf

# Mean of error
average_error = np.mean(errors)
print("Average error: ", average_error)

Average error:  -0.041144822683176596


### Lasso

In [ ]:
lasso = Lasso()

hyper_param_grid_lasso = [{'alpha': [0.001, 0.005, 0.01, 0.03]}]


lasso_gs = GridSearchCV(lasso, hyper_param_grid_lasso, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

lasso_gs.fit(X_train, y_train)
lasso_gs.best_params_


{'alpha': 0.01}

In [ ]:
best_model_lasso = lasso_gs.best_estimator_
best_model_lasso

Lasso(alpha=0.01)

In [ ]:
# Measuring from validation
y_pred_lasso = best_model_lasso.predict(X_val)
mse_lasso = mean_squared_error(y_val, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)
print("Lasso:")
print("RMSE: ", rmse_lasso)
print("")

Lasso:
RMSE:  0.6413255257621376



In [ ]:
# Measuring from kfold crossvalidation
np.sqrt(-lasso_gs.best_score_)

0.6644945493553975

In [ ]:
# Errors
errors = y_val - y_pred_lasso

# Mean of error
average_error = np.mean(errors)
print("Average error: ", average_error)

Average error:  -0.03874406588710667


### Ridge

In [ ]:
ridge = Ridge()

hyper_param_grid_ridge = [{'alpha': [5.0, 5.5, 6.0, 7.0, 8.0]}]


ridge_gs = GridSearchCV(ridge, hyper_param_grid_ridge, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

ridge_gs.fit(X_train, y_train)
ridge_gs.best_params_

{'alpha': 5.0}

In [ ]:
best_model_ridge = ridge_gs.best_estimator_
best_model_ridge

Ridge(alpha=5.0)

In [ ]:
# Measuring from validation
y_pred_ridge = best_model_ridge.predict(X_val)
mse_ridge = mean_squared_error(y_val, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)
print("Ridge:")
print("RMSE: ", rmse_ridge)
print("")

Ridge:
RMSE:  0.6412936572006167



In [ ]:
# Measuring from kfold crossvalidation
np.sqrt(-ridge_gs.best_score_)

0.6647303633208083

In [ ]:
# Errors
errors = y_val - y_pred_ridge

# Mean of error
average_error = np.mean(errors)
print("Average error: ", average_error)

Average error:  -0.03738875452463976


### GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor(random_state=42)

hyper_param_grid_gbr = [{
    'n_estimators': [65, 70, 75],
    'learning_rate': [5.0, 0.1, 0.01],
    'max_depth': [6, 7, 8]
  }]


gbr_gs = GridSearchCV(gbr, hyper_param_grid_gbr, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

gbr_gs.fit(X_train, y_train)
gbr_gs.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 75}

In [ ]:
best_model_gbr = gbr_gs.best_estimator_
best_model_gbr

GradientBoostingRegressor(max_depth=6, n_estimators=75, random_state=42)

In [ ]:
# Measuring from validation
y_pred_gbr = best_model_gbr.predict(X_val)
mse_gbr = mean_squared_error(y_val, y_pred_gbr)
rmse_gbr = np.sqrt(mse_gbr)
print("Gradient Boosting:")
print("RMSE: ", rmse_gbr)
print("")

Gradient Boosting:
RMSE:  0.5751387314224466



In [ ]:
# Measuring from kfold crossvalidation
np.sqrt(-gbr_gs.best_score_)

0.5920375343553939

In [ ]:
# Errors
errors = y_val - y_pred_gbr

# Mean of error
average_error = np.mean(errors)
print("Average error: ", average_error)

Average error:  -0.031357246209342916


### SVR

In [ ]:
svr = SVR()

hyper_param_grid_svr = [{
        'C': [5.0, 7.0, 10.0],
        'kernel': ['linear', 'rbf']
    }]



svr_gs = GridSearchCV(svr, hyper_param_grid_svr, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

svr_gs.fit(X_train, y_train)
svr_gs.best_params_

{'C': 5.0, 'kernel': 'rbf'}

In [ ]:
best_model_svr = svr_gs.best_estimator_
best_model_svr

SVR(C=5.0)

In [ ]:
# Measuring from validation
y_pred_svr = best_model_svr.predict(X_val)
mse_svr = mean_squared_error(y_val, y_pred_svr)
rmse_svr = np.sqrt(mse_svr)
print("SVR:")
print("RMSE: ", rmse_svr)
print("")

SVR:
RMSE:  0.6231267241017862



In [ ]:
# Measuring from kfold crossvalidation
np.sqrt(-svr_gs.best_score_)

0.6210887832476177

In [ ]:
# Errors
errors = y_val - y_pred_svr

# Mean of error
average_error = np.mean(errors)
print("Average error: ", average_error)


Average error:  0.004381962351660666


### XGBRegressor

In [ ]:
xgb = XGBRegressor(random_state=42)

hyper_param_grid_xgb = [{
        'n_estimators': [180, 200, 230, 250],
        'learning_rate': [0.1, 0.05, 0.01, 0.001],
        'max_depth': [6, 7, 8, 10]
    }]


xgb_gs = GridSearchCV(xgb, hyper_param_grid_xgb, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

xgb_gs.fit(X_train, y_train)
xgb_gs.best_params_

{'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 250}

In [ ]:
best_model_xgb = xgb_gs.best_estimator_
best_model_xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [ ]:
# Measuring from validation
y_pred_xgb = best_model_xgb.predict(X_val)
mse_xgb = mean_squared_error(y_val, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
print("XGBoost:")
print("RMSE: ", rmse_xgb)
print("")

XGBoost:
RMSE:  0.5644203168461571



In [ ]:
# Measuring from kfold crossvalidation
np.sqrt(-xgb_gs.best_score_)

0.5856633569932852

In [ ]:
# Beräkna felet
errors = y_val - y_pred_xgb

# Medelvärde av felet (genomsnittligt fel i kronor)
average_error = np.mean(errors)
print("Average error: ", average_error)

Average error:  -0.02972291095142702


### DecisionTree

In [ ]:
dt = DecisionTreeRegressor(random_state=42)

hyper_param_grid_dt = [{
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }]


dt_gs = GridSearchCV(dt, hyper_param_grid_dt, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

dt_gs.fit(X_train, y_train)
dt_gs.best_params_

{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}

In [ ]:
best_model_dt = dt_gs.best_estimator_
best_model_dt

DecisionTreeRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                      random_state=42)

In [ ]:
# Measuring from validation
y_pred_dt = best_model_dt.predict(X_val)
mse_dt = mean_squared_error(y_val, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)
print("Decision Tree:")
print("RMSE: ", rmse_dt)
print("")

Decision Tree:
RMSE:  0.6585644165913727



In [ ]:
# Measuring from kfold crossvalidation
np.sqrt(-dt_gs.best_score_)

0.6481478349426931

In [ ]:
# Beräkna felet
errors = y_val - y_pred_dt

# Medelvärde av felet (genomsnittligt fel i kronor)
average_error = np.mean(errors)
print("Average error: ", average_error)


Average error:  -0.032085782420418214


## The best models seem too be:
- Lasso 
- Ridge 
- DecisionTree

It could perhaps point to a fact that the model works better when less complex, since the models which performed in the best three were Lasso and Ridge and they work with either taking away features which are not important(put them to 0, Lasso) or mkes them even less relevant(puts there values closer to 0, Ridge)

They have similar testresults.